In [1]:
from google.colab import drive
drive.mount('/content/drive')

rawdata_path = '/content/drive/MyDrive/my_project/my_project1/rawdata'
data_path = '/content/drive/MyDrive/my_project/my_project1/data'

# rawdata_path = 'rawdata/'
# data_path = 'data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

import re
import os

import geopandas as gpd

# 제출용 코드에서 깔끔하게 보이게,..
import warnings
warnings.filterwarnings("ignore")

In [3]:
change = pd.read_excel(f'{rawdata_path}/부동산통계/전국지가변동률조사_202402/02.지가변동률/지가변동률_월별_읍면동별.xlsx', header = 10)
change.head()

,지 역,Unnamed: 1,Unnamed: 2,Unnamed: 3,2021년 01월,Unnamed: 5,2021년 02월,Unnamed: 7,2021년 03월,Unnamed: 9,...,2023년 10월,Unnamed: 71,2023년 11월,Unnamed: 73,2023년 12월,Unnamed: 75,2024년 01월,Unnamed: 77,2024년 02월,Unnamed: 79
0,NaN,NaN,NaN,NaN,변동률,누계,변동률,누계,변동률,누계,...,변동률,누계,변동률,누계,변동률,누계,변동률,누계,변동률,누계
1,전 국,NaN,NaN,NaN,0.311,0.311,0.311,0.623,0.338,0.963,...,0.158,0.516,0.157,0.674,0.148,0.823,0.131,0.131,0.134,0.265
2,대도시,NaN,NaN,NaN,0.375,0.375,0.373,0.749,0.39,1.142,...,0.189,0.525,0.179,0.705,0.159,0.865,0.135,0.135,0.133,0.267
3,시지역,NaN,NaN,NaN,0.255,0.255,0.257,0.513,0.292,0.806,...,0.136,0.551,0.145,0.697,0.148,0.845,0.138,0.138,0.146,0.284
4,군지역,NaN,NaN,NaN,0.205,0.205,0.202,0.408,0.239,0.648,...,0.064,0.203,0.067,0.271,0.065,0.336,0.059,0.059,0.06,0.119


In [4]:
change.rename(columns = {'지 역' : '시도',
                       'Unnamed: 1' : '시군구',
                       'Unnamed: 2' : '미상',
                       'Unnamed: 3' : '읍면동'}, inplace = True)
change.drop(columns = '미상', inplace = True)
change.head()

,시도,시군구,읍면동,2021년 01월,Unnamed: 5,2021년 02월,Unnamed: 7,2021년 03월,Unnamed: 9,2021년 04월,...,2023년 10월,Unnamed: 71,2023년 11월,Unnamed: 73,2023년 12월,Unnamed: 75,2024년 01월,Unnamed: 77,2024년 02월,Unnamed: 79
0,NaN,NaN,NaN,변동률,누계,변동률,누계,변동률,누계,변동률,...,변동률,누계,변동률,누계,변동률,누계,변동률,누계,변동률,누계
1,전 국,NaN,NaN,0.311,0.311,0.311,0.623,0.338,0.963,0.35,...,0.158,0.516,0.157,0.674,0.148,0.823,0.131,0.131,0.134,0.265
2,대도시,NaN,NaN,0.375,0.375,0.373,0.749,0.39,1.142,0.409,...,0.189,0.525,0.179,0.705,0.159,0.865,0.135,0.135,0.133,0.267
3,시지역,NaN,NaN,0.255,0.255,0.257,0.513,0.292,0.806,0.301,...,0.136,0.551,0.145,0.697,0.148,0.845,0.138,0.138,0.146,0.284
4,군지역,NaN,NaN,0.205,0.205,0.202,0.408,0.239,0.648,0.237,...,0.064,0.203,0.067,0.271,0.065,0.336,0.059,0.059,0.06,0.119


In [5]:
change['시도'].fillna(method='ffill', inplace = True)
change['시군구'].fillna(method='ffill', inplace = True)
change = change[change['시도'] == '서울특별시']
change = change[change['읍면동'].notna()]
change.head()

,시도,시군구,읍면동,2021년 01월,Unnamed: 5,2021년 02월,Unnamed: 7,2021년 03월,Unnamed: 9,2021년 04월,...,2023년 10월,Unnamed: 71,2023년 11월,Unnamed: 73,2023년 12월,Unnamed: 75,2024년 01월,Unnamed: 77,2024년 02월,Unnamed: 79
9,서울특별시,종로구,청운동,0.382,0.382,0.384,0.768,0.436,1.206,0.412,...,0.282,-0.069,0.354,0.285,0.248,0.533,0.186,0.186,0.189,0.374
10,서울특별시,종로구,신교동,0.382,0.382,0.384,0.768,0.436,1.206,0.412,...,0.282,-0.069,0.354,0.285,0.248,0.533,0.186,0.186,0.189,0.374
11,서울특별시,종로구,궁정동,0.382,0.382,0.384,0.768,0.436,1.206,0.412,...,0.282,-0.069,0.354,0.285,0.248,0.533,0.186,0.186,0.189,0.374
12,서울특별시,종로구,효자동,0.391,0.391,0.417,0.81,0.402,1.215,0.412,...,0.27,0.399,0.346,0.747,0.234,0.982,0.175,0.175,0.252,0.428
13,서울특별시,종로구,창성동,0.391,0.391,0.417,0.81,0.402,1.215,0.412,...,0.27,0.399,0.346,0.747,0.234,0.982,0.175,0.175,0.252,0.428


In [6]:
change_year = ['2021', '2022', '2023']
change_var = ['지가변동률_2022', '지가변동률_2023', '지가변동률_2024']

for idx, year in enumerate(change_year):
    change[change_var[idx]] = change[[col for col in change.columns if year in col]].sum(axis=1)



In [7]:
change = change[['시군구', '읍면동'] + change_var ]
change.head()

,시군구,읍면동,지가변동률_2022,지가변동률_2023,지가변동률_2024
9,종로구,청운동,4.865,7.383,7.917
10,종로구,신교동,4.865,7.383,7.917
11,종로구,궁정동,4.865,7.383,7.917
12,종로구,효자동,4.855,8.173,9.152
13,종로구,창성동,4.855,8.173,9.152


In [8]:
rate = pd.read_excel(f'{rawdata_path}/부동산통계/전국지가변동률조사_202402/01.지가지수/지가지수_월별_읍면동별.xlsx', header = 10)
rate.head()

,지 역,Unnamed: 1,Unnamed: 2,Unnamed: 3,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,...,2023년 05월,2023년 06월,2023년 07월,2023년 08월,2023년 09월,2023년 10월,2023년 11월,2023년 12월,2024년 01월,2024년 02월
0,전 국,NaN,NaN,NaN,73.034,73.083,73.162,73.259,73.354,73.432,...,99.193,99.243,99.306,99.411,99.538,99.695,99.852,100,100.131,100.265
1,대도시,NaN,NaN,NaN,69.538,69.592,69.671,69.779,69.878,69.959,...,99.089,99.14,99.212,99.323,99.475,99.663,99.841,100,100.135,100.267
2,시지역,NaN,NaN,NaN,76.03,76.071,76.15,76.238,76.334,76.408,...,99.24,99.295,99.354,99.464,99.573,99.708,99.852,100,100.138,100.284
3,군지역,NaN,NaN,NaN,80.051,80.104,80.179,80.261,80.324,80.384,...,99.705,99.723,99.737,99.76,99.804,99.868,99.935,100,100.059,100.119
4,수도권,NaN,NaN,NaN,71.182,71.214,71.288,71.382,71.483,71.551,...,98.951,99.017,99.099,99.239,99.405,99.609,99.812,100,100.167,100.34


In [9]:
rate.rename(columns = {'지 역' : '시도',
                       'Unnamed: 1' : '시군구',
                       'Unnamed: 2' : '미상',
                       'Unnamed: 3' : '읍면동'}, inplace = True)
rate.drop(columns = '미상', inplace = True)
rate.head()

,시도,시군구,읍면동,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,...,2023년 05월,2023년 06월,2023년 07월,2023년 08월,2023년 09월,2023년 10월,2023년 11월,2023년 12월,2024년 01월,2024년 02월
0,전 국,NaN,NaN,73.034,73.083,73.162,73.259,73.354,73.432,73.445,...,99.193,99.243,99.306,99.411,99.538,99.695,99.852,100,100.131,100.265
1,대도시,NaN,NaN,69.538,69.592,69.671,69.779,69.878,69.959,69.947,...,99.089,99.14,99.212,99.323,99.475,99.663,99.841,100,100.135,100.267
2,시지역,NaN,NaN,76.03,76.071,76.15,76.238,76.334,76.408,76.45,...,99.24,99.295,99.354,99.464,99.573,99.708,99.852,100,100.138,100.284
3,군지역,NaN,NaN,80.051,80.104,80.179,80.261,80.324,80.384,80.423,...,99.705,99.723,99.737,99.76,99.804,99.868,99.935,100,100.059,100.119
4,수도권,NaN,NaN,71.182,71.214,71.288,71.382,71.483,71.551,71.553,...,98.951,99.017,99.099,99.239,99.405,99.609,99.812,100,100.167,100.34


In [10]:
rate['시도'].fillna(method='ffill', inplace = True)
rate['시군구'].fillna(method='ffill', inplace = True)
rate = rate[rate['시도'] == '서울특별시']
rate = rate[rate['읍면동'].notna()]
rate.drop(columns = '시도', inplace = True)
rate.head()

,시군구,읍면동,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,...,2023년 05월,2023년 06월,2023년 07월,2023년 08월,2023년 09월,2023년 10월,2023년 11월,2023년 12월,2024년 01월,2024년 02월
8,종로구,청운동,-,-,-,-,-,-,-,-,...,98.91,98.929,98.957,98.99,99.121,99.401,99.753,100,100.186,100.374
9,종로구,신교동,-,-,-,-,-,-,-,-,...,98.91,98.929,98.957,98.99,99.121,99.401,99.753,100,100.186,100.374
10,종로구,궁정동,-,-,-,-,-,-,-,-,...,98.91,98.929,98.957,98.99,99.121,99.401,99.753,100,100.186,100.374
11,종로구,효자동,-,-,-,-,-,-,-,-,...,98.81,98.862,98.891,98.959,99.155,99.423,99.767,100,100.175,100.428
12,종로구,창성동,-,-,-,-,-,-,-,-,...,98.81,98.862,98.891,98.959,99.155,99.423,99.767,100,100.175,100.428


In [11]:
df = pd.merge(change, rate,
              how = 'left',
              on = ['시군구', '읍면동'])

In [12]:
df['key'] = df.apply(lambda row: row['시군구'] + ' ' + row['읍면동'], axis=1)
# df = df[['key'] + var_year]
df.reset_index(drop = True, inplace = True)
df.head()

,시군구,읍면동,지가변동률_2022,지가변동률_2023,지가변동률_2024,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,...,2023년 06월,2023년 07월,2023년 08월,2023년 09월,2023년 10월,2023년 11월,2023년 12월,2024년 01월,2024년 02월,key
0,종로구,청운동,4.865,7.383,7.917,-,-,-,-,-,...,98.929,98.957,98.99,99.121,99.401,99.753,100,100.186,100.374,종로구 청운동
1,종로구,신교동,4.865,7.383,7.917,-,-,-,-,-,...,98.929,98.957,98.99,99.121,99.401,99.753,100,100.186,100.374,종로구 신교동
2,종로구,궁정동,4.865,7.383,7.917,-,-,-,-,-,...,98.929,98.957,98.99,99.121,99.401,99.753,100,100.186,100.374,종로구 궁정동
3,종로구,효자동,4.855,8.173,9.152,-,-,-,-,-,...,98.862,98.891,98.959,99.155,99.423,99.767,100,100.175,100.428,종로구 효자동
4,종로구,창성동,4.855,8.173,9.152,-,-,-,-,-,...,98.862,98.891,98.959,99.155,99.423,99.767,100,100.175,100.428,종로구 창성동


In [13]:
dong_convert = pd.read_csv(f'{data_path}/region_code_all.csv')
dong_convert

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2
0,11010720.0,1111051500,1111010100,종로구,청운효자동,청운효자동,청운동,청운동,종로구 청운효자동,종로구 청운효자동
1,11010720.0,1111051500,1111010200,종로구,청운효자동,청운효자동,신교동,신교동,종로구 청운효자동,종로구 청운효자동
2,11010720.0,1111051500,1111010300,종로구,청운효자동,청운효자동,궁정동,궁정동,종로구 청운효자동,종로구 청운효자동
3,11010720.0,1111051500,1111010400,종로구,청운효자동,청운효자동,효자동,효자동,종로구 청운효자동,종로구 청운효자동
4,11010720.0,1111051500,1111010500,종로구,청운효자동,청운효자동,창성동,창성동,종로구 청운효자동,종로구 청운효자동
...,...,...,...,...,...,...,...,...,...,...
736,11250660.0,1174065000,1174010800,강동구,성내2동,성내제2동,성내동,성내동,강동구 성내2동,강동구 성내제2동
737,11250670.0,1174066000,1174010800,강동구,성내3동,성내제3동,성내동,성내동,강동구 성내3동,강동구 성내제3동
738,11250740.0,1174068500,1174010500,강동구,길동,길동,길동,길동,강동구 길동,강동구 길동
739,11250700.0,1174069000,1174010600,강동구,둔촌1동,둔촌제1동,둔촌동,둔촌동,강동구 둔촌1동,강동구 둔촌제1동


In [14]:
dong_convert['key'] = dong_convert['시군구'] + ' ' + dong_convert['법정동1']
dong_convert.head()

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2,key
0,11010720.0,1111051500,1111010100,종로구,청운효자동,청운효자동,청운동,청운동,종로구 청운효자동,종로구 청운효자동,종로구 청운동
1,11010720.0,1111051500,1111010200,종로구,청운효자동,청운효자동,신교동,신교동,종로구 청운효자동,종로구 청운효자동,종로구 신교동
2,11010720.0,1111051500,1111010300,종로구,청운효자동,청운효자동,궁정동,궁정동,종로구 청운효자동,종로구 청운효자동,종로구 궁정동
3,11010720.0,1111051500,1111010400,종로구,청운효자동,청운효자동,효자동,효자동,종로구 청운효자동,종로구 청운효자동,종로구 효자동
4,11010720.0,1111051500,1111010500,종로구,청운효자동,청운효자동,창성동,창성동,종로구 청운효자동,종로구 청운효자동,종로구 창성동


In [15]:
temp = pd.merge(dong_convert, df,
                     how = 'left',
                     on = ['시군구', 'key'])
temp.reset_index(inplace = True)
temp.head()

,index,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,...,2023년 05월,2023년 06월,2023년 07월,2023년 08월,2023년 09월,2023년 10월,2023년 11월,2023년 12월,2024년 01월,2024년 02월
0,0,11010720.0,1111051500,1111010100,종로구,청운효자동,청운효자동,청운동,청운동,종로구 청운효자동,...,98.91,98.929,98.957,98.99,99.121,99.401,99.753,100,100.186,100.374
1,1,11010720.0,1111051500,1111010200,종로구,청운효자동,청운효자동,신교동,신교동,종로구 청운효자동,...,98.91,98.929,98.957,98.99,99.121,99.401,99.753,100,100.186,100.374
2,2,11010720.0,1111051500,1111010300,종로구,청운효자동,청운효자동,궁정동,궁정동,종로구 청운효자동,...,98.91,98.929,98.957,98.99,99.121,99.401,99.753,100,100.186,100.374
3,3,11010720.0,1111051500,1111010400,종로구,청운효자동,청운효자동,효자동,효자동,종로구 청운효자동,...,98.81,98.862,98.891,98.959,99.155,99.423,99.767,100,100.175,100.428
4,4,11010720.0,1111051500,1111010500,종로구,청운효자동,청운효자동,창성동,창성동,종로구 청운효자동,...,98.81,98.862,98.891,98.959,99.155,99.423,99.767,100,100.175,100.428


In [16]:
selected_month = '01월'
selected_year = ['2022', '2023', '2024']
var_year = ['지가_' + year for year in selected_year]

rate_month = [col for col in rate.columns.tolist() if selected_month in col]
rate_year = [col for col in rate_month if any(year in col for year in selected_year)]

In [17]:
final = temp.groupby(['ADM_CD','key1'])[change_var + rate_year].mean().reset_index()
final = final[['ADM_CD']+ change_var + rate_year]
final.columns = [['ADM_CD']+ change_var + var_year]
final

,ADM_CD,지가변동률_2022,지가변동률_2023,지가변동률_2024,지가_2022,지가_2023,지가_2024
0,11010530.0,4.8585,7.902167,8.764667,96.517917,98.973167,100.18275
1,11010540.0,4.673571,7.840571,8.613,96.461429,99.057857,100.133714
2,11010550.0,5.295,8.004,9.032,96.764,98.822,100.209
3,11010560.0,5.151,8.009,9.204,96.442,98.655,100.254
4,11010570.0,4.891,6.939,7.421,97.809,99.397,100.152
...,...,...,...,...,...,...,...
421,11250730.0,5.853,9.443,10.696,95.619,98.748,100.18
422,11250740.0,5.343,7.85,8.613,97.15,99.205,100.145
423,11250750.0,4.596,6.481,7.182,97.736,99.244,100.099
424,11250760.0,4.719,6.897,7.601,97.45,99.239,100.086


In [18]:
final.isna().sum().sum()

0

In [19]:
final.to_csv(f'{data_path}/final/real.csv', index = False)